# Major League Baseball Strength of Schedule

The 2025 MLB season began on March 25th in Tokyo, Japan when the Chicago Cubs played the defending World Series champions the Los Angeles Dodgers. Starting the season against the defending champs can be a tall task, and unfortunately for Cubs their schedule was expected to continue to be tough well into April and May. The Cubs opponents were a common talking point during Cubs broadcasts. It was often stated that the Cubs had the hardest strength of schedule (SOS) to open the 2025 season. Every time the broadcast began talking on SOS, I would ask myself a few questions: 
1. How is strengh of schedule measured
2. How much harder is the Cubs schedule
3. How does the 2025 season compare to previous seasons

These questions led me to develop the following research questions
1. Opponents previous season win percentage
2. Opponents previous season run differential
3. Who had the hardest 2025 opening schedule? What about easiest
4. Distribution of previous years opponents win% 

In [46]:
import pandas as pd
import numpy as np

df_game_data = pd.read_csv("./clean_data/game_results.csv").drop(columns=['H/A', 'win_loss', 'game_time', 'day_night', 'attendance','game_length',])
df_season_data = pd.read_csv("./clean_data/season_results.csv").fillna({'teams': 'LAA'})

## Description of Data

Add description of both tables here

In [47]:
print(f'Dimensions of game data: {df_game_data.shape}')
print(f'Dimensions of season data: {df_season_data.shape}')

Dimensions of game data: (46740, 7)
Dimensions of season data: (330, 7)


There was one blank value in the `df_season_data` caused by Los Angeles Angels because they changed their name

## Merge description

In [57]:
#join data on team and year
# need to offset the season in the game data

df_game_data = df_game_data.assign(previous_season=lambda x: x.season-1) 

df_merge_data = (
    pd.merge(df_game_data, df_season_data.loc[:, ['teams', 'season', 'win_%']], how='left', left_on=['opponent', 'previous_season'], right_on=['teams', 'season'])
    .dropna(subset=['teams'])
    .drop(columns=['season_y', 'previous_season', 'teams'])
    .rename(columns={'win_%': 'opponent_win%', 'season_x': 'season'})
)
#6214 records => good

df_merge_data.head()

,date,team,opponent,runs_scored,runs_allowed,game_number,season,opponent_win%
4858,2016-04-04,ARI,COL,5,10,1,2016,0.420
4859,2016-04-05,ARI,COL,11,6,2,2016,0.420
4860,2016-04-06,ARI,COL,3,4,3,2016,0.420
4861,2016-04-07,ARI,CHC,6,14,4,2016,0.599
4862,2016-04-08,ARI,CHC,3,2,5,2016,0.599


In [69]:
df_merge_data.loc[df_merge_data['game_number']<=40].groupby(['team', 'season']).agg('mean').sort_values(by = 'opponent_win%', ascending=False)

runs_scored  runs_allowed  game_number  opponent_win%
team season                                                       
NYY  2024       4.944444      3.444444    20.055556       0.549722
DET  2023       3.575000      4.575000    20.500000       0.548225
TOR  2022       3.567568      3.864865    21.432432       0.546838
ARI  2018       4.100000      3.400000    20.500000       0.545450
MIL  2021       3.575000      4.000000    20.500000       0.545275
...                  ...           ...          ...            ...
NYY  2019       5.000000      4.100000    20.500000       0.459900
BOS  2025       4.800000      4.350000    20.500000       0.459450
     2019       5.424242      5.000000    21.333333       0.450606
MIN  2020       4.375000      3.500000    20.500000       0.448050
     2021       4.705882      5.205882    19.441176       0.447029

[300 rows x 4 columns]